# Experimentation with HHL quantum circuits and preconditioners
In this notebook, we are conducting experiments using the Harrow–Hassidim–Lloyd (HHL) quantum algorithm, comparing the algorithm's execution time across different sizes of linear systems, both with and without the use of preconditioners.

In [1]:
# Uncomment To install HHL functional lib 
# pip install git+https://github.com/anedumla/quantum_linear_solvers

from linear_solvers import NumPyLinearSolver, HHL
from linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz
from qiskit.quantum_info import Statevector
from scipy.sparse import diags
from scipy.linalg import toeplitz, circulant
import numpy as np
import time

from qiskit.utils import algorithm_globals
algorithm_globals.massive=True

## Utility Functions 
Utility functions for solving a linear system with and without HHL, calculating the condition number of a matrix, extending matrices to Hermitian form, and ensuring dimensions are powers of 2. It is crucial to note that the Qiskit HHL algorithm only supports Hermitian matrices and whose size that is a power of 2.

In [2]:
def generate_tridi(a, b, matrix_size, toeplitz=False):
    if not isinstance(matrix_size, int) or np.log2(matrix_size) % 1 != 0:
        raise ValueError("Matrix dimension must be an integer 2^n.")

    if toeplitz:
        tridi_matrix = TridiagonalToeplitz(np.log2(matrix_size), a, b)
    else:
        tridi_matrix = diags([b, a, b], [-1, 0, 1], shape=(matrix_size, matrix_size)).toarray()

    return tridi_matrix

def solve_hhl(matrix, vector, tol=1e-2, verbose=False):
    matrix = np.array(matrix, dtype=float)
    vector = np.array(vector, dtype=float)
    
    matrix_old = matrix
    matrix = matrix / np.linalg.norm(vector)
    vector = vector / np.linalg.norm(vector)
    n = matrix.shape[0]
    
    start_time_real = time.time()
    start_time = time.process_time()
    hhl = HHL(tol).solve(matrix, vector)
    elapsed_cpu_time = time.process_time() - start_time
    elapsed_real_time = time.time() - start_time_real

    # Get solution vector from state vector
    result_vector = Statevector(hhl.state).data.real
    start_index = int(result_vector.shape[0]/2)
    result_vector = result_vector[start_index : (start_index + n)]
    result_vector = hhl.euclidean_norm * result_vector / np.linalg.norm(result_vector)

    if verbose:
        print(f"Matrix: {matrix_old}")
        print(f"HHL Solution Norm: {hhl.euclidean_norm}")
        print(f"HHL Solution Vector: {result_vector}")
        print(f"CPU Time: {elapsed_cpu_time} seconds")

    return result_vector, elapsed_cpu_time, elapsed_real_time

def solve_classical(matrix, vector, verbose=False):
    matrix = np.array(matrix, dtype=float)
    vector = np.array(vector, dtype=float)

    matrix = matrix / np.linalg.norm(vector)
    vector = vector / np.linalg.norm(vector)

    classical = NumPyLinearSolver().solve(matrix, vector)
    result_vector = classical.state

    if verbose:
        print(f"Classical Solution Norm: {classical.euclidean_norm}")
        print(f"Classical Solution Vector: {result_vector}")

    return result_vector

def error(classical, hhl):
    return np.linalg.norm(classical - hhl)/ np.linalg.norm(classical)

# Add zeros so that matrix is of size 2^n
def extend_matrix_simple(matrix, vector):
    n = matrix.shape[0]
    
    if np.log2(n) % 1 != 0:
        extend_n = int(2**(np.floor(np.log2(n))+1))
    else:
        extend_n = n
        
    new_matrix = np.eye(extend_n)
    new_matrix[:n,:n] = matrix

    new_vector = np.append(vector, [0]*(extend_n - n))

    return new_matrix, new_vector

# Extend to an hermitian matrix
def extend_matrix_hermitian(matrix, vector):
    if not is_hermitian(matrix):
        n = matrix.shape[0]
        extend_n = 2*n

        new_matrix = np.zeros([extend_n, extend_n])
        new_matrix[:n,n:] = np.conj(matrix.T)
        new_matrix[n:,:n] = matrix
        
        new_vector = np.append(np.conj(vector), vector)
        return new_matrix, new_vector
    else:
        return matrix, vector

# Verify if matrix is hermitian
def is_hermitian(matrix):
    return np.allclose(matrix, np.conj(matrix.T))

# Modify matrix and vector so that they can be used on HHL (hermitian and of dimension 2^n)
def set_system(matrix,vector):
    matrix = np.array(matrix, dtype=float)
    vector = np.array(vector, dtype=float)
    n = matrix.shape[0]
    
    if np.log2(n) % 1 != 0: # Matrix is not of size 2^n
        matrix, vector = extend_matrix_simple(matrix, vector)
    if not is_hermitian(matrix): # Matrix is not hermitian
        matrix, vector = extend_matrix_hermitian(matrix,vector)   
    return matrix, vector

def condition_number(matrix):
    matrix = np.array(matrix, dtype=float)
    eigenvalues = np.linalg.eigvals(matrix)
    eigenvalues_abs = np.abs(eigenvalues)
    eigen_max = np.max(eigenvalues_abs)
    eigen_min = np.min(eigenvalues_abs)
    return eigen_max/eigen_min

# Solve linear system without preconditioner and print info (matrix/vector doesn't need to be extended previously)
def solve_wp(matrix,vector, tol=1e-2):
    matrix = np.array(matrix, dtype=float)
    vector = np.array(vector, dtype=float)  
    n = matrix.shape[0]
    cond = condition_number(matrix)
    
    set_matrix1,set_vector1 = set_system(matrix,vector)
    
    # Non preconditioned system resolution
    hhl_result1, cpu_time1, real_time1 = solve_hhl(set_matrix1,set_vector1,tol)
    classical_result1 = solve_classical(set_matrix1,set_vector1)
    hhl_result1, classical_result1 = hhl_result1[:n], classical_result1[:n]
    result_error1 = error(classical_result1, hhl_result1)
    
    print(f"\nSolving system:")
    if set_matrix1.shape[0] > matrix.shape[0]:
        print(f"Warning: System had to be extended to N = {set_matrix1.shape[0]}. Input matrix is not hermitian and/or not of size 2^n. Condition number of extended matrix is actually {condition_number(set_matrix1)}\n")
    print(f"\nCondition Number: {cond}")
    print(f"HHL Result: {hhl_result1}")
    print(f"Classical Result: {classical_result1}")
    print(f"Error: {result_error1}")
    print(f"CPU Time: {cpu_time1}")
    print(f"Wall Time: {real_time1}")

# Solve linear system and print info
def solve(matrix,vector,preconditioner,tol=1e-2):
    matrix = np.array(matrix, dtype=float)
    vector = np.array(vector, dtype=float)  
    n = matrix.shape[0]
    preconditioner = np.array(preconditioner, dtype=float)
    cond_before = condition_number(matrix)
    
    pc_matrix = np.dot(np.linalg.inv(preconditioner), matrix)
    pc_vector = np.dot(np.linalg.inv(preconditioner), vector)
    cond_after = condition_number(pc_matrix)
    
    set_matrix1,set_vector1 = set_system(matrix,vector)
    #print(f"Matrix NPC: {set_matrix1} and vector NPC is {set_vector1}")
    set_matrix2,set_vector2 = set_system(pc_matrix,pc_vector)
    #print(f"Matrix PC: {set_matrix2} and vector PC is {set_vector2}")

    # Non preconditioned system resolution
    hhl_result1, cpu_time1, real_time1 = solve_hhl(set_matrix1,set_vector1,tol)
    classical_result1 = solve_classical(set_matrix1,set_vector1)
    hhl_result1, classical_result1 = hhl_result1[:n], classical_result1[:n]
    result_error1 = error(classical_result1, hhl_result1)
    
    print(f"\nNon preconditioned system:")
    if set_matrix1.shape[0] > matrix.shape[0]:
        print(f"Warning: System had to be extended to N = {set_matrix1.shape[0]}. Input matrix is not hermitian and/or not of size 2^n. Condition number of extended matrix is actually {condition_number(set_matrix1)}")
    print(f"\nCondition Number: {cond_before}")
    print(f"HHL Result: {hhl_result1}")
    print(f"Classical Result: {classical_result1}")
    print(f"Error: {result_error1}")
    print(f"CPU Time: {cpu_time1}")
    print(f"Wall Time: {real_time1}")
    
    # Preconditioned system resolution
    hhl_result2, cpu_time2, real_time2 = solve_hhl(set_matrix2,set_vector2,tol)
    classical_result2 = solve_classical(set_matrix2,set_vector2)
    hhl_result2, classical_result2 = hhl_result2[:n], classical_result2[:n]
    result_error2 = error(classical_result1, hhl_result2) # Result compared to classical result before preconditioning
    
    print(f"\nPreconditioned system:")
    if set_matrix2.shape[0] > matrix.shape[0]:
        print(f"Warning: System had to be extended to N = {set_matrix2.shape[0]}. Input matrix is not hermitian and/or not of size 2^n. Condition number of extended matrix is actually {condition_number(set_matrix2)}")
    print(f"\nCondition Number: {cond_after}")
    print(f"HHL Result: {hhl_result2}")
    print(f"Classical Result: {classical_result2}")
    print(f"Error: {result_error2}")
    print(f"CPU Time: {cpu_time2}")
    print(f"Wall Time: {real_time2}")
    

## Chan's optimal circulant preconditioner for solving Toeplitz systems


Utilizing Chang's circular preconditioner as mentioned in [this article](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.98.062321), which is optimal for Toeplitz systems, we aim to reduce the execution time of the HHL algorithm through system conditioning. Thus, with the circular preconditioner denoted as C, we attempt to solve the following linear system:

$$C^{-1}Ax = C^{-1}b$$

In [3]:
def circulant_preconditioner(toeplitz):
    np.array(toeplitz, dtype=float)
    col = toeplitz[:, 0]
    row = toeplitz[0, :]
    n = toeplitz.shape[0]
    
    col = np.append(col,0)
    i = np.arange(n)
    c = (i * col[n-i] + (n-i) * row[i]) / n
    
    circulant_matrix = circulant(c)
    circulant_matrix = np.array(circulant_matrix, dtype=float)
    return circulant_matrix

## Examples with Hermitian Matrix

Here, the original matrix is hermitian. However, upon the application of the circulant preconditioner, the resulting preconditioned matrix becomes non-Hermitian. Consequently, the execution time is affected as the non-Hermitian system needs to be expanded to meet HHL requirements, and a larger N (system size) inherently leads to slower processing.

### 1st Example: 4x4

In [5]:
col = np.array([1, 2, 3, 4])
rows = np.array([1, 2, 3, 4])
n = 4
toeplitz1 = toeplitz(col, rows)

vector1 = [1]*4
circulant_matrix1 = circulant_preconditioner(toeplitz1)

solve(matrix=toeplitz1, vector=vector1, preconditioner=circulant_matrix1, tol=1e-2)


Non preconditioned system:

Condition Number: 15.532997913802955
HHL Result: [0.19376465 0.00601178 0.00601178 0.19376465]
Classical Result: [0.2 0.  0.  0.2]
Error: 0.04330736616628421
CPU Time: 3.71875
Wall Time: 4.141200542449951

Preconditioned system:

Condition Number: 3.999999999999999
HHL Result: [ 0.20124471 -0.00138801 -0.00138801  0.20124471]
Classical Result: [ 2.00000000e-01 -8.78540656e-17  8.60378948e-17  2.00000000e-01]
Error: 0.009321826162858309
CPU Time: 9.53125
Wall Time: 9.880000829696655


### 2nd Example - 5x5 

In [9]:
col = np.array([1, 2, 3, 4, 5])
rows = np.array([1, 2, 3, 4, 5])
n = 5
toeplitz1 = toeplitz(col, rows)

vector1 = [1]*n
circulant_matrix1 = circulant_preconditioner(toeplitz1)

solve(matrix=toeplitz1, vector=vector1, preconditioner=circulant_matrix1, tol=1e-2)


Non preconditioned system:

Condition Number: 23.71320010708135
HHL Result: [ 0.16380338 -0.00029009  0.00109691 -0.00029009  0.16380338]
Classical Result: [ 1.66666667e-01  2.06877846e-17 -6.20633538e-17  0.00000000e+00
  1.66666667e-01]
Error: 0.017883789341986676
CPU Time: 20.53125
Wall Time: 22.00065517425537

Preconditioned system:

Condition Number: 5.000000000000005
HHL Result: [ 0.15978379 -0.00053029  0.00710488 -0.00053029  0.15978379]
Classical Result: [ 1.66666667e-01 -9.21287682e-18  9.04265670e-17  1.00660260e-17
  1.66666667e-01]
Error: 0.05122709504271788
CPU Time: 141.90625
Wall Time: 147.92321515083313


## Examples with Non-Hermitian Matrix

Here, the original matrix is non-Hermitian, resulting in a preconditioned matrix that is also non-Hermitian. Consequently, enhancements in execution time are more noticeable. Nevertheless, the impact is less apparent in smaller systems, with significant improvements observed primarily for systems of sizes around or exceeding 16x16.

### 1st Example - 4x4
System was expanded to 8x8. A very slight improvement in CPU time is noticeable.

In [10]:
col = [1,2,3,4]
row = [1,3,3,5]
toeplitz2 = toeplitz(col,row)
cond = condition_number(toeplitz2)

print(toeplitz2,cond)

[[1 3 3 5]
 [2 1 3 3]
 [3 2 1 3]
 [4 3 2 1]] 7.143198287855598


In [11]:
circulant_matrix2 = circulant_preconditioner(toeplitz2)

print(circulant_matrix2)

[[1.   2.75 3.   3.25]
 [3.25 1.   2.75 3.  ]
 [3.   3.25 1.   2.75]
 [2.75 3.   3.25 1.  ]]


In [12]:
vector2 = [1]*4
solve(matrix=toeplitz2, vector=vector2, preconditioner=circulant_matrix2, tol=1e-8)


Non preconditioned system:

Condition Number: 7.143198287855598
HHL Result: [ 0.19507743 -0.03126737  0.08893163  0.12750297]
Classical Result: [ 0.1971831  -0.02816901  0.08450704  0.12676056]
Error: 0.02330768423876229
CPU Time: 9.234375
Wall Time: 9.577510356903076

Preconditioned system:

Condition Number: 2.3747540689691475
HHL Result: [ 0.19802748 -0.02654185  0.08154386  0.12743242]
Classical Result: [ 0.1971831  -0.02816901  0.08450704  0.12676056]
Error: 0.01415092101299045
CPU Time: 9.15625
Wall Time: 9.64200496673584


### 2nd Example - 5x5
System was expanded to 16x16. It is possible to notice a very small improvement.

In [12]:
# Example - not hermitian
col = [1,2,3,4,6]
row = [1,3,3,5,6]
toeplitz3 = toeplitz(col,row)

circulant_matrix3 = circulant_preconditioner(toeplitz3)

vector3 = [1]*5
solve(matrix=toeplitz3, vector=vector3, preconditioner=circulant_matrix3, tol=1e-2)


Non preconditioned system:

Condition Number: 10.890012974267385
HHL Result: [ 0.16851603 -0.01831186 -0.05815727  0.03844     0.14075436]
Classical Result: [ 0.18215613 -0.0260223  -0.07434944  0.04089219  0.15241636]
Error: 0.10052881651572633
CPU Time: 89.33305121599997
Wall Time: 84.92791867256165

Preconditioned system:

Condition Number: 3.1496652910584424
HHL Result: [ 0.18150996 -0.02691934 -0.0750809   0.04009067  0.15184238]
Classical Result: [ 0.18215613 -0.0260223  -0.07434944  0.04089219  0.15241636]
Error: 0.006515442038095342
CPU Time: 87.40156941200001
Wall Time: 83.3616726398468


### 3rd Example - 3x3
The system was expanded to 8x8, and the matrix exhibits a slightly more sparse structure, featuring two null diagonals. It is noticeable that this characteristic has nullified the modest improvement observed earlier.

In [5]:
col = [1,2,0]
row = [1,3,0]
toeplitz4 = toeplitz(col,row)

circulant_matrix4 = circulant_preconditioner(toeplitz4)

vector4 = [1]*3
solve(matrix=toeplitz4, vector=vector4, preconditioner=circulant_matrix4, tol=1e-4)


Non preconditioned system:

Condition Number: 4.464101615137749
HHL Result: [-0.09027091  0.36489337  0.27349181]
Classical Result: [-0.09090909  0.36363636  0.27272727]
Error: 0.003459638877289046
CPU Time: 6.921875
Wall Time: 9.20133090019226

Preconditioned system:

Condition Number: 1.0049605160974084
HHL Result: [-0.0979961   0.35802018  0.27229722]
Classical Result: [-0.09090909  0.36363636  0.27272727]
Error: 0.019529275615147737
CPU Time: 7.953125
Wall Time: 9.261417388916016


## Larger Non-hermitian Experiments 

### 1st Example - 16x16 Sparse

This example employs a 16x16 tridiagonal matrix, that is, it is highly sparse. Upon examination, it becomes apparent that despite the preconditioned system improving the condition number, the execution time has deteriorated. This observation leads to the conclusion that sparse matrices are not well-suited for preconditioning (at least in this particular preconditioner case). This is attributed to the transformation of a sparse matrix into a dense one when multiplied by a preconditioner. Such a comparison becomes unequal, as the efficiency of HHL's quantum algorithm relies heavily on the sparsity of the matrices involved.

In [16]:
example = np.diag([2]*16) + np.diag([1/2]*15, k=1) + np.diag([1/0.8]*15, k=-1)
precond_ex = circulant_preconditioner(example)
vector_ex = [1]*16

print(f"Condition number: {condition_number(example)}")

Condition number: 7.972974369262368


In [17]:
precond_matrix = np.dot(np.linalg.inv(precond_ex), example)
precond_vector = np.dot(np.linalg.inv(precond_ex), vector_ex)
print(f"Condition number with precond: {condition_number(precond_matrix)}")

Condition number with precond: 1.5332663414747338


In [18]:
m1, v1 = set_system(example, vector_ex)
m2, v2 = set_system(precond_matrix, precond_vector)
print(f"Condition number of extended matrix: {condition_number(m1)}")
print(f"Condition number of extended precond matrix: {condition_number(m2)}")

Condition number of extended matrix: 12.388294459981728
Condition number of extended precond matrix: 3.845436857038151


In [19]:
solve(example, vector_ex, precond_ex, tol = 1e-2)


Non preconditioned system:

Condition Number: 7.972974369262368
HHL Result: [0.46964315 0.10235569 0.38780072 0.16694204 0.3400583  0.20865954
 0.31184969 0.23281757 0.29469423 0.24600348 0.28228793 0.25334943
 0.27047107 0.26396862 0.24254121 0.3429006 ]
Classical Result: [0.47340137 0.10639454 0.39091844 0.17033989 0.34134433 0.20877295
 0.31154738 0.2318781  0.29361913 0.24582823 0.28263927 0.25487235
 0.27391242 0.26716945 0.24654115 0.34591178]
Error: 0.008835878627672135
CPU Time: 1472.140625
Wall Time: 1527.0267403125763

Preconditioned system:

Condition Number: 1.5332663414747338
HHL Result: [0.47277421 0.10457079 0.38962937 0.16880741 0.34071113 0.20714941
 0.31078787 0.23032109 0.29249079 0.24465651 0.28130441 0.25411007
 0.27243429 0.26604559 0.24466082 0.34470055]
Classical Result: [0.47340137 0.10639454 0.39091844 0.17033989 0.34134433 0.20877295
 0.31154738 0.2318781  0.29361913 0.24582823 0.28263927 0.25487235
 0.27391242 0.26716945 0.24654115 0.34591178]
Error: 0.0044

### 2nd Example - 16x16 Dense

This example employs a 16x16 dense Toeplitz matrix, with elements randomly chosen from the range of -2 to 7. The contrast in performance between non-conditioned and conditioned systems is now significantly more pronounced, affirming our earlier assumptions.

In [ ]:
# # Generating a random 16x16 Toeplitz dense matrix

# from random import randint
# from scipy.sparse import diags

# positions = [i for i in range(-16,17)]
# values = [randint(1,4) for i in range(33)]

# example = diags(values, positions, shape = (16,16)).toarray()
# print(example)

In [4]:
example = np.array([
    [5, 1, 1, 6, 1, 6, 4, 2, -1, 5, 5, 5, -1, 7, 7, 5],
    [-2, 5, 1, 1, 6, 1, 6, 4, 2, -1, 5, 5, 5, -1, 7, 7],
    [7, -2, 5, 1, 1, 6, 1, 6, 4, 2, -1, 5, 5, 5, -1, 7],
    [3, 7, -2, 5, 1, 1, 6, 1, 6, 4, 2, -1, 5, 5, 5, -1],
    [2, 3, 7, -2, 5, 1, 1, 6, 1, 6, 4, 2, -1, 5, 5, 5],
    [5, 2, 3, 7, -2, 5, 1, 1, 6, 1, 6, 4, 2, -1, 5, 5],
    [3, 5, 2, 3, 7, -2, 5, 1, 1, 6, 1, 6, 4, 2, -1, 5],
    [6, 3, 5, 2, 3, 7, -2, 5, 1, 1, 6, 1, 6, 4, 2, -1],
    [2, 6, 3, 5, 2, 3, 7, -2, 5, 1, 1, 6, 1, 6, 4, 2],
    [5, 2, 6, 3, 5, 2, 3, 7, -2, 5, 1, 1, 6, 1, 6, 4],
    [6, 5, 2, 6, 3, 5, 2, 3, 7, -2, 5, 1, 1, 6, 1, 6],
    [4, 6, 5, 2, 6, 3, 5, 2, 3, 7, -2, 5, 1, 1, 6, 1],
    [-2, 4, 6, 5, 2, 6, 3, 5, 2, 3, 7, -2, 5, 1, 1, 6],
    [6, -2, 4, 6, 5, 2, 6, 3, 5, 2, 3, 7, -2, 5, 1, 1],
    [0, 6, -2, 4, 6, 5, 2, 6, 3, 5, 2, 3, 7, -2, 5, 1],
    [-1, 0, 6, -2, 4, 6, 5, 2, 6, 3, 5, 2, 3, 7, -2, 5]],dtype=float)

In [5]:
precond_ex = circulant_preconditioner(example)
vector_ex = [1]*16
print(f"Condition number: {condition_number(example)}")

precond_matrix = np.dot(np.linalg.inv(precond_ex), example)
precond_vector = np.dot(np.linalg.inv(precond_ex), vector_ex)
print(f"Condition number with precond: {condition_number(precond_matrix)}\n")

m1, v1 = set_system(example, vector_ex)
m2, v2 = set_system(precond_matrix, precond_vector)
print(f"Condition number of extended matrix: {condition_number(m1)}")
print(f"Condition number of extended precond matrix: {condition_number(m2)}")

Condition number: 20.931004526427586
Condition number with precond: 2.3278536314082445

Condition number of extended matrix: 63.208012902393484
Condition number of extended precond matrix: 13.81530524443855


In [6]:
solve(example, vector_ex, precond_ex, tol=1e-2)


Non preconditioned system:

Condition Number: 20.931004526427586
HHL Result: [ 0.01422812  0.025456    0.0192649   0.00489215 -0.00903158 -0.00329681
  0.01950822  0.03189928  0.02572489  0.02576888  0.03428114  0.03637148
  0.02925109  0.01716268  0.00583081  0.00248407]
Classical Result: [ 0.01666188  0.02774937  0.02161261  0.0073247  -0.00618548 -0.0002984
  0.02218865  0.03413141  0.02784375  0.02786089  0.03643514  0.03839841
  0.03117803  0.01937855  0.00823093  0.00532442]
Error: 0.1007274673772837
CPU Time: 3660.5
Wall Time: 3880.959947347641

Preconditioned system:

Condition Number: 2.3278536314082445
HHL Result: [ 0.01635865  0.02785575  0.02180759  0.00740385 -0.00609993 -0.00043042
  0.02215646  0.03444357  0.02774806  0.02743981  0.03637366  0.03830829
  0.03092455  0.01948177  0.00813884  0.00525609]
Classical Result: [ 0.01666188  0.02774937  0.02161261  0.0073247  -0.00618548 -0.0002984
  0.02218865  0.03413141  0.02784375  0.02786089  0.03643514  0.03839841
  0.0311

### 3rd Example - 16x16 Dense
In this example, preconditioning reduced the condition number by a larger margin. Thus, the improvement in the execution time of HHL is even more evident.

In [21]:
example = np.array([
    [3., 2., 2., 1., 4., 1., 2., 1., 1., 4., 4., 4., 4., 4., 2., 2.],
    [4., 3., 2., 2., 1., 4., 1., 2., 1., 1., 4., 4., 4., 4., 4., 2.],
    [3., 4., 3., 2., 2., 1., 4., 1., 2., 1., 1., 4., 4., 4., 4., 4.],
    [1., 3., 4., 3., 2., 2., 1., 4., 1., 2., 1., 1., 4., 4., 4., 4.],
    [1., 1., 3., 4., 3., 2., 2., 1., 4., 1., 2., 1., 1., 4., 4., 4.],
    [2., 1., 1., 3., 4., 3., 2., 2., 1., 4., 1., 2., 1., 1., 4., 4.],
    [4., 2., 1., 1., 3., 4., 3., 2., 2., 1., 4., 1., 2., 1., 1., 4.],
    [3., 4., 2., 1., 1., 3., 4., 3., 2., 2., 1., 4., 1., 2., 1., 1.],
    [1., 3., 4., 2., 1., 1., 3., 4., 3., 2., 2., 1., 4., 1., 2., 1.],
    [1., 1., 3., 4., 2., 1., 1., 3., 4., 3., 2., 2., 1., 4., 1., 2.],
    [2., 1., 1., 3., 4., 2., 1., 1., 3., 4., 3., 2., 2., 1., 4., 1.],
    [3., 2., 1., 1., 3., 4., 2., 1., 1., 3., 4., 3., 2., 2., 1., 4.],
    [1., 3., 2., 1., 1., 3., 4., 2., 1., 1., 3., 4., 3., 2., 2., 1.],
    [2., 1., 3., 2., 1., 1., 3., 4., 2., 1., 1., 3., 4., 3., 2., 2.],
    [2., 2., 1., 3., 2., 1., 1., 3., 4., 2., 1., 1., 3., 4., 3., 2.],
    [1., 2., 2., 1., 3., 2., 1., 1., 3., 4., 2., 1., 1., 3., 4., 3.]], dtype=float)

In [22]:
precond_ex = circulant_preconditioner(example)
vector_ex = [1]*16
print(f"Condition number: {condition_number(example)}")

precond_matrix = np.dot(np.linalg.inv(precond_ex), example)
precond_vector = np.dot(np.linalg.inv(precond_ex), vector_ex)
print(f"Condition number with precond: {condition_number(precond_matrix)}\n")

m1, v1 = set_system(example, vector_ex)
m2, v2 = set_system(precond_matrix, precond_vector)
print(f"Condition number of extended matrix: {condition_number(m1)}")
print(f"Condition number of extended precond matrix: {condition_number(m2)}")

Condition number: 99.80192726002834
Condition number with precond: 5.52282826373228

Condition number of extended matrix: 214.86127395127494
Condition number of extended precond matrix: 35.40120917007131


In [23]:
solve(example, vector_ex, precond_ex, tol=1e-2)

MemoryError: Unable to allocate 16.0 GiB for an array with shape (32768, 32768) and data type complex128

## 32x32 Non-hermitian Experiment

This experiment, using a 32x32 matrix that was later expanded to a 64x64 system, encountered memory issues and proved unfeasible for execution. The tests were conducted on a computer with 32 GB of RAM, but the HHL execution appears to exceed the available memory capacity. This suggests that Qiskit's HHL may have limited scalability.

In [6]:
# Random Toeplitz

from random import randint
from scipy.sparse import diags
np.set_printoptions(threshold=np.inf)

# positions = [i for i in range(-32,33)]
# values = [randint(1,4) for i in range(65)]

# example = diags(values, positions, shape = (32,32)).toarray()
# print(example)

In [84]:
example = [[2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 1, 2, 1, 3, 4, 4,
  3, 1, 1, 3, 1, 3, 1, 3],
 [3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 1, 2, 1, 3, 4,
  4, 3, 1, 1, 3, 1, 3, 1],
 [2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 1, 2, 1, 3,
  4, 4, 3, 1, 1, 3, 1, 3],
 [4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 1, 2, 1,
  3, 4, 4, 3, 1, 1, 3, 1],
 [4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 1, 2,
  1, 3, 4, 4, 3, 1, 1, 3],
 [1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2, 1,
  2, 1, 3, 4, 4, 3, 1, 1],
 [1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2, 2,
  1, 2, 1, 3, 4, 4, 3, 1],
 [3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3, 2,
  2, 1, 2, 1, 3, 4, 4, 3],
 [2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1, 3,
  2, 2, 1, 2, 1, 3, 4, 4],
 [1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1, 1,
  3, 2, 2, 1, 2, 1, 3, 4],
 [4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1, 1,
  1, 3, 2, 2, 1, 2, 1, 3],
 [3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4, 1,
  1, 1, 3, 2, 2, 1, 2, 1],
 [3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2, 4,
  1, 1, 1, 3, 2, 2, 1, 2],
 [4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1, 2,
  4, 1, 1, 1, 3, 2, 2, 1],
 [2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2, 1,
  2, 4, 1, 1, 1, 3, 2, 2],
 [3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3, 2,
  1, 2, 4, 1, 1, 1, 3, 2],
 [1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3, 3,
  2, 1, 2, 4, 1, 1, 1, 3],
 [1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1, 3,
  3, 2, 1, 2, 4, 1, 1, 1],
 [2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1, 1,
  3, 3, 2, 1, 2, 4, 1, 1],
 [2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3, 1,
  1, 3, 3, 2, 1, 2, 4, 1],
 [4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2, 3,
  1, 1, 3, 3, 2, 1, 2, 4],
 [1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2, 2,
  3, 1, 1, 3, 3, 2, 1, 2],
 [1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2, 2,
  2, 3, 1, 1, 3, 3, 2, 1],
 [3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3, 2,
  2, 2, 3, 1, 1, 3, 3, 2],
 [3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2, 3,
  2, 2, 2, 3, 1, 1, 3, 3],
 [4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4, 2,
  3, 2, 2, 2, 3, 1, 1, 3],
 [4, 4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4, 4,
  2, 3, 2, 2, 2, 3, 1, 1],
 [4, 4, 4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1, 4,
  4, 2, 3, 2, 2, 2, 3, 1],
 [1, 4, 4, 4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1, 1,
  4, 4, 2, 3, 2, 2, 2, 3],
 [2, 1, 4, 4, 4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3, 1,
  1, 4, 4, 2, 3, 2, 2, 2],
 [2, 2, 1, 4, 4, 4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2, 3,
  1, 1, 4, 4, 2, 3, 2, 2],
 [3, 2, 2, 1, 4, 4, 4, 3, 3, 1, 1, 4, 2, 2, 1, 1, 3, 2, 4, 3, 3, 4, 1, 2,
  3, 1, 1, 4, 4, 2, 3, 2]]

In [85]:
example = np.array(example, dtype=np.float64)

In [86]:
precond_ex = circulant_preconditioner(example)
vector_ex = [1]*example.shape[0]
print(f"Condition number: {condition_number(example)}")

precond_matrix = np.dot(np.linalg.inv(precond_ex), example)
precond_vector = np.dot(np.linalg.inv(precond_ex), vector_ex)
print(f"Condition number with precond: {condition_number(precond_matrix)}\n")

m1, v1 = set_system(example, vector_ex)
m2, v2 = set_system(precond_matrix, precond_vector)
print(f"Condition number of extended matrix: {condition_number(m1)}")
print(f"Condition number of extended precond matrix: {condition_number(m2)}")

Condition number: 80.49451967664568
Condition number with precond: 2.6150760336623886

Condition number of extended matrix: 144.41092784911405
Condition number of extended precond matrix: 22.27462236010347


In [ ]:
solve(example, vector_ex, precond_ex, tol = 1e-2)

## Strang's circulant preconditioner for solving Toeplitz systems
This is a simple test employing Strang's circulant preconditioned obtained from the previously mentioned article. While there was an enhancement in execution time, it's noteworthy that the preconditioner, though effective, is not optimal and fails to improve the condition number for any Toeplitz matrix, unlike Chang's. Consequently, Chang's preconditioner appears to be notably more advantageous.

In [ ]:
def circulant_preconditioner_strang(toeplitz):
    np.array(toeplitz, dtype=float)
    col = toeplitz[:, 0]
    row = toeplitz[0, :]
    n = toeplitz.shape[0]
    m = np.floor(n/2).astype(int)
    
    i = np.arange(n)
    c = np.concatenate((col[0:m+1], row[n-m-1:0:-1]))
    
    circulant_matrix = circulant(c)
    circulant_matrix = np.array(circulant_matrix, dtype=float)
    return circulant_matrix

In [94]:
col = [1,2,0,0]
row = [1,5,0,0]
matrix = toeplitz(col,row)

    
print(f"Matrix: {matrix}")
print(f"Condition number: {condition_number(matrix)}")

Matrix: [[1 5 0 0]
 [2 1 5 0]
 [0 2 1 5]
 [0 0 2 1]]
Condition number: 6.408952530039636


In [ ]:
circulant_matrix = circulant_preconditioner_strang(matrix)
precond_matrix = np.dot(np.linalg.inv(circulant_matrix), matrix)
print(f"Preconditioned matrix: {precond_matrix}")
print(f"Condition number with precond: {condition_number(precond_matrix)}")

Preconditioned matrix: [[-1.14583333e-01  5.55111512e-17 -5.55111512e-17 -7.91666667e-02]
 [ 3.02083333e-01  1.00000000e+00  0.00000000e+00 -4.45833333e-01]
 [ 3.85416667e-01 -9.71445147e-17  1.00000000e+00  1.20833333e-01]
 [-1.97916667e-01 -5.55111512e-17 -1.38777878e-17  1.15416667e+00]]
Condition number with precond: 9.197643874566463


In [ ]:
vector = [1]*4
precond_vector = np.dot(np.linalg.inv(circulant_matrix), vector)
print(precond_vector)

[0.125 0.125 0.125 0.125]


In [ ]:
m1, v1 = set_system(matrix, vector)
m2, v2 = set_system(precond_matrix, precond_vector)
print(f"Condition number of extended matrix: {condition_number(m1)}")
print(f"Condition number of extended precond matrix: {condition_number(m2)}")

Condition number of extended matrix: 14.359720869846758
Condition number of extended precond matrix: 12.061167467025179


In [ ]:
solve(matrix,vector,circulant_matrix)


Non preconditioned system:

Condition Number: 6.408952530039636
HHL Result: [-1.04784262  0.39421996  0.52155696 -0.08165526]
Classical Result: [-1.04225352  0.4084507   0.53521127 -0.07042254]
Error: 0.01880802265861577
CPU Time: 21.984531668
Wall Time: 19.0487539768219

Preconditioned system:

Condition Number: 9.197643874566463
HHL Result: [-1.04309798  0.4067096   0.53363124 -0.07168134]
Classical Result: [-1.04225352  0.4084507   0.53521127 -0.07042254]
Error: 0.002250923191235293
CPU Time: 20.859336497999998
Wall Time: 17.735612869262695
